In [4]:
from yaml import safe_load, dump

In [5]:

def read_species(block, nranges):
    species = {}
    
    species['name'] = block[0][:17].strip()
    species['ref'] = block[0][17:].strip()
    
    species['datecode'] = block[1][4:10].strip()
    species['chemform'] = block[1][11:51].strip()
    species['gasOrCond'] = int(block[1][51:53].strip())
    species['molecWeight'] = float(block[1][53:66].strip())
    species['heatOfForm'] = float(block[1][66:].strip())
    
    species['ranges'] = {}
    
    for i in range(nranges):
        Trange = tuple([float(j) for j in block[2+3*i][:22].split()])
        
        exponents = [float(block[2+3*i][24+5*j:29+5*j].strip()) for j in range(7)]
        dh0 = float(block[2+3*i][65:80])
        coeffs1 = [block[3+3*i][16*j:16+16*j].strip() for j in range(5)]
        coeffs2 = [block[4+3*i][16*j:16+16*j].strip() for j in range(2)]
        coeffs3 = [block[4+3*i][48+16*j:16+48+16*j].strip() for j in range(2)]
        
        coeffs = coeffs1 + coeffs2 + coeffs3
        coeffs = [float(i.replace('D','E')) for i in coeffs]
        
        species['ranges'][Trange] = {
            'exp': exponents,
            'coeffs': coeffs,
            'dh0': dh0
        }
        
    return species
        
        
def read_inp(content):
    split_content = content.split('\n')
    if split_content[0].startswith('thermo'):
        trimContent = split_content[2:]
    else:
        trimContent = split_content
        
    blocks = {}
    ranges = {}
    species = {}
    
    while len(trimContent) > 0:
        name = trimContent[0][:16].strip()
        nranges = int(trimContent[1][:2].strip())
        blocklength = 2 + 3*nranges
        blocks[name] = trimContent[:blocklength]
        ranges[name] = nranges
        del trimContent[:blocklength]
        if trimContent[0].startswith('END'): del trimContent[0]
        if trimContent[0] == '': del trimContent[0]
    
    for name, block in blocks.items():
        species[name] = read_species(block, ranges[name])
    
    return species


def load_and_save_inp(filedir, loadname, savename):
    with open(filedir + loadname, 'r') as f:
        content = f.read()
        file = read_inp(content)
        f.close()
        
    fileStr = """
############################################################
#                                                          #
#     Gaseous Species Class and NASA-9 Data                #
#     Author: Wyatt Giroux                                 #
#     Date: 1/10/25                                        #
#                                                          #
############################################################

class Species:
    def __init__(self, species):
        self.name = species['name']
        self.ref = species['ref'] # reference info in NASA-9 data
        self.datecode = species['datecode'] # NASA-9 date code
        self.chemform = species['chemform'] # raw string of chemical formula
        self.gasorcond = species['gasOrCond'] # 0 if gas; non-0 otherwise 
        self.molecweight = species['molecWeight'] # molecular weight [g/mol]
        self.heatofform = species['heatOfForm'] # heat of formation [J/mol]
        self.ranges = species['ranges'] # Dictionary of NASA-9 coefficients


    def __str__(self):
        printstr = ''
        printstr = printstr + f'Species: {self.name}\\n'
        printstr = printstr + f'> Reference: {self.ref}\\n'
        printstr = printstr + f'> Date Code: {self.datecode}\\n'
        printstr = printstr + f'> Molecular Weight: {self.molecweight}\\n'
        printstr = printstr + f'> Heat of Formation: {self.heatofform}\\n'
        printstr = printstr + f'> Range Data: {self.ranges}\\n'
        return printstr
"""
    for species, data in file.items():
        if '-' in species:
            species = species.replace('-','_')
        if ',' in species:
            species = species.replace(',','_')
        if '(' in species:
            species = species.replace('(','_')
        if ')' in species:
            species = species.replace(')','_')

        fileStr = fileStr + '''
{species} = Species({data})
'''.format(species=species, data=data)

    with open(filedir + savename, 'w') as saveFile:
        saveFile.write(fileStr)
        saveFile.close()

In [6]:
load_and_save_inp('./src/data/', 'thermo.inp', 'thermo.py')